In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import json
import os
import requests
import psycopg
import sqlite3
from sqlalchemy import create_engine

dotenv.load_dotenv()

comgresskey = os.getenv('congresskey')
postgres_password = os.getenv('POSTGRES_PASSWORD')


In [2]:
ct = contrans()

In [3]:
dbserver, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD, create_contrans=True)

In [4]:
members = ct.get_bioguideIDs() # members from Congress API
members = ct.make_cand_table(members) # joining the contributions ID with the Congress API data
terms, members = ct.terms_df(members) # seperate the terms (non-atomic) data from members
ideology = ct.get_ideology() # get the ideologial data from voteview.com
ct.make_members_df(members, ideology, engine) # join members and ideologial and upload to postgres database

In [5]:
ct.make_terms_df(terms, engine)

In [6]:
votes = ct.get_votes()
ct.make_votes_df(votes, engine)

In [14]:
# Create voting affinity table
myquery = """
SELECT 
    a.icpsr AS icpsr1, 
    b.icpsr AS icpsr2, 
    AVG(CAST((a.cast_code = b.cast_code) AS INT)) AS agree
FROM votes a
INNER JOIN votes b
ON a.chamber = b.chamber
AND a.rollnumber = b.rollnumber
WHERE a.icpsr=14854 AND b.icpsr!=14854
GROUP BY a.icpsr, b.icpsr
ORDER BY agree DESC
"""
pd.read_sql(myquery, con=engine)

,icpsr1,icpsr2,agree
0,14854,21108,0.932651
1,14854,22378,0.927928
2,14854,29323,0.922421
3,14854,22124,0.913043
4,14854,22336,0.905371
...,...,...,...
443,14854,21566,0.271952
444,14854,21172,0.246914
445,14854,29573,0.230196
446,14854,22106,0.217391


In [17]:
myquery = """
SELECT m.name, m.partyname, m.state, m.district, v.agree
FROM members m
INNER JOIN (
    SELECT 
    a.icpsr AS icpsr1, 
    b.icpsr AS icpsr2, 
    AVG(CAST((a.cast_code = b.cast_code) AS INT)) AS agree
    FROM votes a
    INNER JOIN votes b
    ON a.chamber = b.chamber
    AND a.rollnumber = b.rollnumber
    WHERE a.icpsr=14854 AND b.icpsr!=14854
    GROUP BY a.icpsr, b.icpsr
    ORDER BY agree DESC
    ) v
ON CAST(m.icpsr AS INT) = v.icpsr2
WHERE icpsr is not null
ORDER BY v.agree DESC
"""
pd.read_sql(myquery, con=engine)

,name,partyname,state,district,agree
0,"Womack, Steve",Republican,AR,03,0.932651
1,"Fong, Vince",Republican,CA,20,0.927928
2,"Calvert, Ken",Republican,CA,41,0.922421
3,"Hinson, Ashley",Republican,IA,02,0.913043
4,"Kiley, Kevin",Republican,CA,03,0.905371
...,...,...,...,...,...
433,"Sablan, Gregorio Kilili Camacho",Democratic,MP,S,0.298507
434,"Bowman, Jamaal",Democratic,NY,16,0.286445
435,"Evans, Dwight",Democratic,PA,03,0.271952
436,"Bush, Cori",Democratic,MO,01,0.217391


In [9]:
myquery = """
SELECT *
FROM (SELECT 
    chamber, 
    rollnumber, 
    icpsr, 
    cast_code
FROM votes
WHERE icpsr=14854)
"""
pd.read_sql(myquery, con=engine)

,chamber,rollnumber,icpsr,cast_code
0,House,1,14854,6
1,House,2,14854,6
2,House,3,14854,6
3,House,4,14854,6
4,House,5,14854,6
...,...,...,...,...
1168,House,1160,14854,1
1169,House,1162,14854,1
1170,House,1164,14854,1
1171,House,1170,14854,1
